In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload

Here we graph the probability of a pair of sets with Jacquard distance $p$ ending up in the list of final comparisons:

$$f(p) = 1-(1-p^k)^b$$

We also graph the probability threshold, which is the inflection point in this function. It is given by:

$$\frac{d^2f}{dp^2} = 0$$
$$\left( \frac{(b-1)k}{k-1} + 1\right)^{-1/k} \approx b^{-1/k}$$


In [2]:
b = 7
k = 5
f = lambda p: 1 - (1 - p**k)**b
threshold = ((b-1)*k/float(k-1)+1)**(-1.0/k)
print threshold, b**(-1.0/k)

x = np.linspace(0, 1, 100)
y = f(x)
plt.plot(x, y)
plt.axvline(x=threshold)
plt.show()

0.651802789631 0.6776109134


In [78]:
import set_lsh as s
reload(s)
minhash = s.make_minhash(s.hash_family(s.default_gen, 10))

In [82]:
import hashlib
a = hashlib.md5
b = default_gen()

%timeit a('12345').hexdigest

%timeit a('12345').hexdigest

The slowest run took 12.47 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 402 ns per loop
